In [1]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv(r"D:\jupyter projects\Machine Learning\Column Transfer\covid_toy.csv")
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [6]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],
                                                test_size=0.2)

In [8]:
X_train

,age,gender,fever,cough,city
91,38,Male,NaN,Mild,Delhi
46,19,Female,101.0,Mild,Mumbai
24,13,Female,100.0,Strong,Kolkata
8,19,Female,100.0,Strong,Bangalore
0,60,Male,103.0,Mild,Kolkata
...,...,...,...,...,...
22,71,Female,98.0,Strong,Kolkata
96,51,Female,101.0,Strong,Kolkata
58,23,Male,98.0,Strong,Mumbai
60,24,Female,102.0,Strong,Bangalore


## Without Column Transfer (the hard way)

In [16]:
#we have null values in fever column so first we need to fill them 
from sklearn.impute import SimpleImputer

si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])
X_test_fever = si.transform(X_test[['fever']])
                                 
X_train_fever.shape

(80, 1)

In [17]:
# Ordinalencoding -> cough
from sklearn.preprocessing import OrdinalEncoder
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# also the test data
X_test_cough = oe.transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

In [19]:
# OneHotEncoding -> gender,city

from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(drop='first', sparse_output=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# also the test data
X_test_gender_city = ohe.transform(X_test[['gender','city']])

X_train_gender_city.shape

(80, 4)

In [14]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [24]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
# also the test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed.shape

(80, 7)

## Using the Column Transfer(Easy Way) 

In [25]:
from sklearn.compose import ColumnTransformer

In [27]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse_output=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [28]:
transformer.fit_transform(X_train).shape

(80, 7)